# data importation

Not to run

In [ ]:
"""
import os
import pandas as pd

# Path alla cartella contenente i file .story
folder_path = "./stories"

# Lista per memorizzare i dati estratti
data = []

# Controllare se la cartella esiste
if os.path.exists(folder_path):
    # Iterare sui file nella cartella
    for filename in os.listdir(folder_path):
        if filename.endswith(".story"):
            file_path = os.path.join(folder_path, filename)

            # Leggere il contenuto del file (esempio per file di testo)
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()

            # Aggiungere i dati in formato dizionario
            data.append({"filename": filename, "content": content})

    # Creare il DataFrame
    df = pd.DataFrame(data)

    # Visualizzare i primi 5 record
    print(df.head())

    # Salvare il DataFrame in un file CSV (opzionale)
    df.to_csv("stories_dataframe.csv", index=False)
    print("Il DataFrame è stato salvato in 'stories_dataframe.csv'")
else:
    print(f"La cartella '{folder_path}' non esiste. Controlla il percorso.")


# MODEL

In [ ]:
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/TEXT_M_ PROJECT/stories_dataframe.csv')

In [ ]:
!pip install rouge-score
!pip install -q contractions rouge
!pip install -q ipdb

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import networkx as nx
from rouge_score import rouge_scorer
import nltk
import re
import contractions
from tqdm import tqdm
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from textwrap import wrap
import gensim
import difflib
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from rouge import Rouge

# Scaricare risorse NLTK necessarie
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## DIVIDE HIGHLIGHTS AND STORY

In [ ]:
def preprocess_text_with_highlights(file_content):

    # Dividi il testo in sezioni usando \n\n@highlight\n\n
    sections = file_content.split("\n\n@highlight\n\n")

    # Il primo elemento è il corpo principale
    main_text = sections[0].strip()

    # I successivi elementi sono gli highlights
    highlights = [section.strip() for section in sections[1:]]

    # Dividi il corpo principale in righe
    main_text_lines = main_text.split("\n")

    # Pulisci le righe del corpo principale
    main_text_cleaned = "\n".join(line.strip() for line in main_text_lines if line.strip())

    # Preprocessa gli highlights per rimuovere righe vuote
    highlights_cleaned = [highlight.strip() for highlight in highlights if highlight.strip()]

    return main_text_cleaned, highlights_cleaned

In [ ]:
df = df.sample(n=20000, random_state=42)

In [ ]:
df['story'], df['highlights'] = zip(*df['content'].map(preprocess_text_with_highlights))
df= df[['story','highlights']]
df

,story,highlights
67822,OTTAWA (CNN) -- Canadian officials say they ha...,[Canadian officials say they were following up...
75462,(CNN) -- Rick Perry's decision to pull out of ...,"[Paul Sracic: If Mitt Romney wins in S.C., the..."
74989,Comedian Joan Rivers isn't about to apologize ...,[Rivers made the quip Monday on a show about O...
78677,(CNN) -- A Southern California man who attract...,"[Sloan Steven Briles, 35, of Irvine, Californi..."
57651,(CNN) -- Verizon pulled a rabbit out of its co...,"[Writers: Verizon, AT&T deals are attempts to ..."
...,...,...
53135,"Baghdad (CNN) -- Wearing hooded sweatshirts, b...",[Smashing Hits is an English-language rap grou...
82993,(CNN) -- At least 150 people drowned when a bo...,[Thousands of refugees from fighting in Libya ...
28719,(CNN) -- Athletic Bilbao striker Fernando Llor...,[Juventus open talks with Athletic Bilbao's Fe...
77930,"(CNN)""Let us build a fairyland for the people ...","[""Let the strong wind of fish farming blow acr..."


# PREPROCESS

In [ ]:


# Funzione per creare stopwords personalizzate
def create_stopwords(lst_langs=["english"], lst_add_words=[], lst_keep_words=[]):
    lst_stopwords = set()
    for lang in lst_langs:
        lst_stopwords = lst_stopwords.union(set(nltk.corpus.stopwords.words(lang)))
    lst_stopwords = lst_stopwords.union(lst_add_words)
    lst_stopwords = list(set(lst_stopwords) - set(lst_keep_words))
    return sorted(list(set(lst_stopwords)))

lst_stopwords = create_stopwords(
    lst_langs=['english'],
    lst_add_words=["cnn", "say", "said", "new", "wa", "ha"]
)

# Funzione avanzata per preprocessare il testo
def utils_preprocess_text(txt, lst_regex=None, punkt=True, lower=True, slang=True, lst_stopwords=None, stemm=False, lemm=True):
    if lst_regex is not None:
        for regex in lst_regex:
            txt = re.sub(regex, '', txt)

    txt = re.sub(r'[^\w\s\.]', '', txt) if punkt is True else txt
    txt = " ".join([word.strip() for word in txt.split()])
    txt = txt.lower() if lower is True else txt
    txt = contractions.fix(txt) if slang is True else txt

    lst_txt = txt.split()

    if stemm is True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_txt = [ps.stem(word) for word in lst_txt]

    if lemm is True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_txt = [lem.lemmatize(word) for word in lst_txt]

    if lst_stopwords is not None:
        lst_txt = [word for word in lst_txt if word not in lst_stopwords]

    txt = " ".join(lst_txt)
    return txt

# Funzione per aggiungere una colonna di testo preprocessato
def add_preprocessed_text(data, column, lst_regex=None, punkt=False, lower=False, slang=False, lst_stopwords=None, stemm=False, lemm=False, remove_na=True):
    dtf = data.copy()

    dtf = dtf.dropna(subset=[column])
    tqdm.pandas(desc='Cleaning text')
    dtf[column + "_clean"] = dtf[column].progress_apply(lambda x: utils_preprocess_text(x, lst_regex, punkt, lower, slang, lst_stopwords, stemm, lemm))

    tqdm.pandas(desc='Checking Residuals')
    dtf["check"] = dtf[column + "_clean"].progress_apply(lambda x: len(x))
    if dtf["check"].min() == 0:
        print("--- found NAs ---")
        print(dtf[[column, column + "_clean"]][dtf["check"] == 0].head())
        if remove_na is True:
            dtf = dtf[dtf["check"] > 0]

    return dtf.drop("check", axis=1)

# Applica il preprocessing alla colonna 'story'
df = add_preprocessed_text(
    data=df,
    column='story',
    punkt=True,
    lower=True,
    slang=True,
    lst_stopwords=lst_stopwords,
    stemm=False,
    lemm=True
)

Checking Residuals: 100%|██████████| 20000/20000 [00:00<00:00, 445557.92it/s]

--- found NAs ---
      story story_clean
8256                   
75850                  
50916                  
68517                  
63345                  


In [ ]:
# Funzione per preprocessare highlights
def preprocess_highlights(data, column, lst_regex=None, punkt=False, lower=False, slang=False, lst_stopwords=None, stemm=False, lemm=True):
    def process_list(highlight_list):
        if isinstance(highlight_list, list):
            return [utils_preprocess_text(h, lst_regex, punkt, lower, slang, lst_stopwords, stemm, lemm) for h in highlight_list]
        return []

    data[column + "_clean"] = data[column].apply(process_list)
    return data

# Applica il preprocessing a 'highlights'
df = preprocess_highlights(
    data=df,
    column='highlights',
    punkt=True,
    lower=True,
    slang=True,
    lst_stopwords=lst_stopwords,
    stemm=False,
    lemm=True
)

# LSA not preprocessed

In [ ]:


# 1. Preprocessing Function
def preprocess_text(text):
    # Tokenizza il testo in frasi
    sentences = sent_tokenize(text)
    preprocessed_sentences = []

    # Stopwords e rimozione punteggiatura
    stop_words = set(stopwords.words('english'))
    for sentence in sentences:
        # Rimuovi punteggiatura e metti in minuscolo
        sentence = re.sub(r'[^\w\s]', '', sentence).lower()
        # Rimuovi stopwords
        words = [word for word in sentence.split() if word not in stop_words]
        preprocessed_sentences.append(" ".join(words))

    return preprocessed_sentences

# 2. Compute TF-IDF Matrix
def compute_tfidf(corpus, max_features=5000):
    vectorizer = TfidfVectorizer(max_features=max_features, stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names_out()
    return tfidf_matrix, feature_names

# 3. Perform LSA
def perform_lsa(tfidf_matrix, n_components=5):
    lsa = TruncatedSVD(n_components=n_components, random_state=42)
    lsa_matrix = lsa.fit_transform(tfidf_matrix)
    return lsa, lsa_matrix

# 4. Score Sentences
def score_sentences(lsa_matrix):
    sentence_scores = np.sum(lsa_matrix ** 2, axis=1)  # Ponderazione basata sull'importanza del topic
    return sentence_scores

# 5. Generate Summary
def generate_summary(sentences, scores, top_n=3):
    top_indices = np.argsort(scores)[-top_n:][::-1]  # Indici delle frasi con i punteggi più alti
    summary = [sentences[i] for i in top_indices]
    return summary  # Restituisce una lista di frasi

# 6. Summarize Document
def summarize_text(df, column_name="stories", n_components=5, top_n=3):
    summaries = []
    for story in df[column_name]:
        # Controlla storie vuote o non valide
        if not isinstance(story, str) or not story.strip():
            summaries.append([])  # Aggiungi un sommario vuoto
            continue

        # Preprocessing
        sentences = sent_tokenize(story)
        preprocessed_sentences = preprocess_text(story)

        # Compute TF-IDF
        tfidf_matrix, _ = compute_tfidf(preprocessed_sentences)

        # Apply LSA
        lsa, lsa_matrix = perform_lsa(tfidf_matrix, n_components=n_components)

        # Score Sentences
        scores = score_sentences(lsa_matrix)

        # Generate Summary
        summary = generate_summary(sentences, scores, top_n=top_n)
        summaries.append(summary)

    return summaries




In [ ]:
df['summary'] = summarize_text(df, column_name='story', n_components=5, top_n=3)

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


# LSA PREPROCESSED


In [ ]:
# 2. Compute TF-IDF Matrix
def compute_tfidf(corpus, max_features=5000):
    vectorizer = TfidfVectorizer(max_features=max_features, stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names_out()
    return tfidf_matrix, feature_names

# 3. Perform LSA
def perform_lsa(tfidf_matrix, n_components=5):
    lsa = TruncatedSVD(n_components=n_components, random_state=42)
    lsa_matrix = lsa.fit_transform(tfidf_matrix)
    return lsa, lsa_matrix

# 4. Score Sentences
def score_sentences(lsa_matrix):
    sentence_scores = np.sum(lsa_matrix ** 2, axis=1)  # Ponderazione basata sull'importanza del topic
    return sentence_scores

# 5. Generate Summary
def generate_summary(sentences, scores, top_n=3):
    top_indices = np.argsort(scores)[-top_n:][::-1]  # Indici delle frasi con i punteggi più alti
    summary = [sentences[i] for i in top_indices]
    return summary  # Restituisce una lista di frasi

# 6. Summarize Document
def summarize_text(df, column_name="stories", n_components=5, top_n=3):
    summaries = []
    for story in df[column_name]:
        # Controlla storie vuote o non valide
        if not isinstance(story, str) or not story.strip():
            summaries.append([])  # Aggiungi un sommario vuoto
            continue

        # Preprocessing
        sentences = sent_tokenize(story)
        preprocessed_sentences = preprocess_text(story)

        # Compute TF-IDF
        tfidf_matrix, _ = compute_tfidf(preprocessed_sentences)

        # Apply LSA
        lsa, lsa_matrix = perform_lsa(tfidf_matrix, n_components=n_components)

        # Score Sentences
        scores = score_sentences(lsa_matrix)

        # Generate Summary
        summary = generate_summary(sentences, scores, top_n=top_n)
        summaries.append(summary)

    return summaries


In [ ]:
df['summary'] = summarize_text(df, column_name='story_clean', n_components=5, top_n=3)

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


### EVALUATION

In [ ]:
# Funzione per calcolare i punteggi ROUGE
def evaluate_summary_row(ground_truth_list, predicted_list):
    rouge = Rouge()
    # Verifica se le liste sono vuote
    if not ground_truth_list or not predicted_list:
        return {
            'rouge-1': 0.0,
            'rouge-2': 0.0,
            'rouge-l': 0.0
        }
    # Concatena le liste in stringhe
    ground_truth_text = " ".join(ground_truth_list)
    predicted_text = " ".join(predicted_list)
    scores = rouge.get_scores(predicted_text, ground_truth_text, avg=True)
    return {
        'rouge-1': scores['rouge-1']['f'],
        'rouge-2': scores['rouge-2']['f'],
        'rouge-l': scores['rouge-l']['f']
    }


In [ ]:
# Applicare la funzione riga per riga con lambda
df['rouge_scores'] = df.apply(lambda row: evaluate_summary_row(row['highlights'], row['summary']), axis=1)

# Estrarre i valori medi dei punteggi ROUGE per valutazione complessiva
df['rouge-1'] = df['rouge_scores'].apply(lambda x: x['rouge-1'])
df['rouge-2'] = df['rouge_scores'].apply(lambda x: x['rouge-2'])
df['rouge-l'] = df['rouge_scores'].apply(lambda x: x['rouge-l'])

avg_rouge_1 = df['rouge-1'].mean()
avg_rouge_2 = df['rouge-2'].mean()
avg_rouge_l = df['rouge-l'].mean()

avg_model_score = np.mean([avg_rouge_1, avg_rouge_2, avg_rouge_l])

# Stampare i risultati
print("DataFrame con valutazioni ROUGE:")
print(df)
print(f"\nValutazione complessiva del modello:")
print(f"ROUGE-1: {avg_rouge_1:.2f}, ROUGE-2: {avg_rouge_2:.2f}, ROUGE-L: {avg_rouge_l:.2f}, AVG ROUGE: {avg_model_score:.2f}")

DataFrame con valutazioni ROUGE:
                                                   story  \
67822  OTTAWA (CNN) -- Canadian officials say they ha...   
75462  (CNN) -- Rick Perry's decision to pull out of ...   
74989  Comedian Joan Rivers isn't about to apologize ...   
78677  (CNN) -- A Southern California man who attract...   
57651  (CNN) -- Verizon pulled a rabbit out of its co...   
...                                                  ...   
53135  Baghdad (CNN) -- Wearing hooded sweatshirts, b...   
82993  (CNN) -- At least 150 people drowned when a bo...   
28719  (CNN) -- Athletic Bilbao striker Fernando Llor...   
77930  (CNN)"Let us build a fairyland for the people ...   
54147  (CNN) -- Australia has a new prime minister. A...   

                                              highlights  \
67822  [Canadian officials say they were following up...   
75462  [Paul Sracic: If Mitt Romney wins in S.C., the...   
74989  [Rivers made the quip Monday on a show about O...   
78677 

### eval with preprocess

In [ ]:
# Applicare la funzione riga per riga con lambda
df['rouge_scores'] = df.apply(lambda row: evaluate_summary_row(row['highlights_clean'], row['summary']), axis=1)

# Estrarre i valori medi dei punteggi ROUGE per valutazione complessiva
df['rouge-1'] = df['rouge_scores'].apply(lambda x: x['rouge-1'])
df['rouge-2'] = df['rouge_scores'].apply(lambda x: x['rouge-2'])
df['rouge-l'] = df['rouge_scores'].apply(lambda x: x['rouge-l'])

avg_rouge_1 = df['rouge-1'].mean()
avg_rouge_2 = df['rouge-2'].mean()
avg_rouge_l = df['rouge-l'].mean()

avg_model_score = np.mean([avg_rouge_1, avg_rouge_2, avg_rouge_l])

# Stampare i risultati
print("DataFrame con valutazioni ROUGE:")
print(df)
print(f"\nValutazione complessiva del modello:")
print(f"ROUGE-1: {avg_rouge_1:.2f}, ROUGE-2: {avg_rouge_2:.2f}, ROUGE-L: {avg_rouge_l:.2f}, AVG ROUGE: {avg_model_score:.2f}")

DataFrame con valutazioni ROUGE:
                                                   story  \
67822  OTTAWA (CNN) -- Canadian officials say they ha...   
75462  (CNN) -- Rick Perry's decision to pull out of ...   
74989  Comedian Joan Rivers isn't about to apologize ...   
78677  (CNN) -- A Southern California man who attract...   
57651  (CNN) -- Verizon pulled a rabbit out of its co...   
...                                                  ...   
53135  Baghdad (CNN) -- Wearing hooded sweatshirts, b...   
82993  (CNN) -- At least 150 people drowned when a bo...   
28719  (CNN) -- Athletic Bilbao striker Fernando Llor...   
77930  (CNN)"Let us build a fairyland for the people ...   
54147  (CNN) -- Australia has a new prime minister. A...   

                                              highlights  \
67822  [Canadian officials say they were following up...   
75462  [Paul Sracic: If Mitt Romney wins in S.C., the...   
74989  [Rivers made the quip Monday on a show about O...   
78677 


# PAGE RANK with poorli preprocessed text
---



In [ ]:

# Preprocessing: rimuovi valori nulli e normalizza il testo
df = df.dropna(subset=['story', 'highlights'])
df['story'] = df['story'].str.strip()
def preprocess_highlights(value):
    if isinstance(value, str):
        try:
            return eval(value)  # Converte stringhe di liste in liste
        except Exception as e:
            print(f"Errore durante l'elaborazione di {value}: {e}")
            return []  # Ritorna una lista vuota in caso di errore
    elif isinstance(value, list):
        return value  # Ritorna il valore se è già una lista
    else:
        return []  # Gestisce altri tipi non previsti

df['highlights'] = df['highlights'].apply(preprocess_highlights)  # Converte stringhe di liste in liste

# Funzione per suddividere un articolo in frasi
def split_sentences(article):
    from nltk.tokenize import sent_tokenize
    return sent_tokenize(article)

# modello
def generate_summary(article, num_sentences=3):
    if not article or not isinstance(article, str) or article.strip() == "":
        return ["Error: Empty or invalid article"]

    # Suddividi l'articolo in frasi
    sentences = split_sentences(article)

    # Verifica se ci sono sufficienti frasi per il riassunto
    if len(sentences) <= num_sentences:
        return sentences

    try:
        # Calcola la matrice TF-IDF
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(sentences)

        # Calcola la matrice di similarità
        similarity_matrix = cosine_similarity(tfidf_matrix)

        # Crea il grafo e applica PageRank
        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph)

        # Ordina le frasi in base ai punteggi di PageRank
        ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

        # Seleziona le frasi migliori
        summary = [ranked_sentences[i][1] for i in range(num_sentences)]
        return summary
    except Exception as e:
        return [f"Error summarizing article: {str(e)}"]

# Applica il modello a ogni articolo usando apply
df['summary'] = df['story'].apply(lambda x: generate_summary(x))

# Valutazione con ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

references = [" ".join(highlight) for highlight in df['highlights']]
predictions = [" ".join(summary) for summary in df['summary']]

rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
for ref, pred in zip(references, predictions):
    scores = scorer.score(ref, pred)
    rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
    rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

# Stampa la media dei punteggi
print("ROUGE Scores:")
for key in rouge_scores:
    avg_score = sum(rouge_scores[key]) / len(rouge_scores[key])
    print(f"{key}: {avg_score}")

# Salva il risultato in un file CSV
#output_path = "extractive_summarized_articles_with_scores.csv"
#fd.to_csv(output_path, index=False)



ROUGE Scores:
rouge1: 0.27215052308948545
rouge2: 0.08112924833779075
rougeL: 0.16828586256383182


In [ ]:
df

,story,highlights,story_clean,highlights_clean,summary,rouge_scores,rouge-1,rouge-2,rouge-l
67822,OTTAWA (CNN) -- Canadian officials say they ha...,[Canadian officials say they were following up...,ottawa canadian official arrested man convicte...,"[canadian official following tip made arrest, ...","[Dyer was in custody in Montreal Wednesday, an...","{'rouge-1': 0.4347826037334594, 'rouge-2': 0.1...",0.434783,0.127660,0.391304
75462,(CNN) -- Rick Perry's decision to pull out of ...,"[Paul Sracic: If Mitt Romney wins in S.C., the...",rick perry decision pull republican primary so...,[paul sracic mitt romney win s.c. bandwagon ef...,[The Romney campaign would surely be happy at ...,"{'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0}",0.000000,0.000000,0.000000
74989,Comedian Joan Rivers isn't about to apologize ...,[Rivers made the quip Monday on a show about O...,comedian joan river apologize joke made holoca...,"[river made quip monday show oscar fashion, us...","[2 it is about the Holocaust., This is the way...","{'rouge-1': 0.21276595282933464, 'rouge-2': 0....",0.212766,0.000000,0.212766
78677,(CNN) -- A Southern California man who attract...,"[Sloan Steven Briles, 35, of Irvine, Californi...",southern california man attracted widespread a...,[sloan steven briles 35 irvine california char...,[Deputies had to drag him from the tour boat t...,"{'rouge-1': 0.18604650676041115, 'rouge-2': 0....",0.186047,0.000000,0.186047
57651,(CNN) -- Verizon pulled a rabbit out of its co...,"[Writers: Verizon, AT&T deals are attempts to ...",verizon pulled rabbit corporate hat earlier mo...,[writer verizon att deal attempt get spectrum ...,"[The third option, and probably the best under...","{'rouge-1': 0.1538461497961868, 'rouge-2': 0.0...",0.153846,0.048780,0.102564
...,...,...,...,...,...,...,...,...,...
53135,"Baghdad (CNN) -- Wearing hooded sweatshirts, b...",[Smashing Hits is an English-language rap grou...,baghdad wearing hooded sweatshirt baseball hat...,[smashing hit englishlanguage rap group based ...,[The reason for that choice is that they want ...,"{'rouge-1': 0.21739129934782622, 'rouge-2': 0....",0.217391,0.043478,0.173913
82993,(CNN) -- At least 150 people drowned when a bo...,[Thousands of refugees from fighting in Libya ...,least 150 people drowned boat leaving libya ca...,[thousand refugee fighting libya tried flee eu...,"[""Desperate passengers rushed to one side, see...","{'rouge-1': 0.3333333283401921, 'rouge-2': 0.0...",0.333333,0.034483,0.185185
28719,(CNN) -- Athletic Bilbao striker Fernando Llor...,[Juventus open talks with Athletic Bilbao's Fe...,athletic bilbao striker fernando llorente open...,[juventus open talk athletic bilbaos fernando ...,[The Pamplona-born star has been in and out of...,"{'rouge-1': 0.33846153360473374, 'rouge-2': 0....",0.338462,0.029412,0.215385
77930,"(CNN)""Let us build a fairyland for the people ...","[""Let the strong wind of fish farming blow acr...",cnnlet u build fairyland people dint science o...,[let strong wind fish farming blow across coun...,[Published by the state-controlled KCNA news a...,"{'rouge-1': 0.2112676037442968, 'rouge-2': 0.1...",0.211268,0.177215,0.211268


In [ ]:
# Funzione per evidenziare le frasi estratte nella story
def highlight_summary(story, summary):
    highlighted_story = story
    for sentence in summary:
        highlighted_story = highlighted_story.replace(sentence, f"**{sentence}**")
    return highlighted_story

# Applica la funzione per evidenziare le frasi estratte
df['highlighted_story'] = df.apply(lambda row: highlight_summary(row['story'], row['summary']), axis=1)


# PAGE RANK WITH BETTER PREPROCESS
---



In [ ]:


# Funzione per suddividere un articolo in frasi
def split_sentences(article):
    from nltk.tokenize import sent_tokenize
    return sent_tokenize(article)

# Modello per generare riassunti
def generate_summary(article, num_sentences=3):
    if not article or not isinstance(article, str) or article.strip() == "":
        return ["Error: Empty or invalid article"]

    # Suddividi l'articolo in frasi
    sentences = split_sentences(article)

    # Verifica se ci sono sufficienti frasi per il riassunto
    if len(sentences) <= num_sentences:
        return sentences

    try:
        # Calcola la matrice TF-IDF
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(sentences)

        # Calcola la matrice di similarità
        similarity_matrix = cosine_similarity(tfidf_matrix)

        # Crea il grafo e applica PageRank
        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph)

        # Ordina le frasi in base ai punteggi di PageRank
        ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

        # Seleziona le frasi migliori
        summary = [ranked_sentences[i][1] for i in range(num_sentences)]
        return summary
    except Exception as e:
        return [f"Error summarizing article: {str(e)}"]

# Applica il modello a ogni articolo usando apply
df['summary'] = df['story_clean'].apply(lambda x: generate_summary(x))



In [ ]:
df.iloc[12]['highlights']

['President Obama ordered flags lowered to half-staff in memory of Nelson Mandela',
 'Mandela loomed large over American activists and politicians',
 'His plight has been repeatedly depicted by Hollywood']

In [ ]:
df.iloc[12]['summary']

['mandela released south african government began transition think felt like part skinner said.',
 'word nelson mandela death spread quickly across united state bringing mix reverence grief man born south africa end belonged world.',
 'honor step shoe nelson mandela portray man defied odds broke barrier championed human right eye world elba said.']

In [ ]:
references = [" ".join(highlight) for highlight in df['highlights_clean']]
predictions = [" ".join(summary) for summary in df['summary']]

rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
for ref, pred in zip(references, predictions):
    scores = scorer.score(ref, pred)
    rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
    rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

# Stampa la media dei punteggi
print("ROUGE Scores:")
for key in rouge_scores:
    avg_score = sum(rouge_scores[key]) / len(rouge_scores[key])
    print(f"{key}: {avg_score}")


ROUGE Scores:
rouge1: 0.26559843850110454
rouge2: 0.08976203866551674
rougeL: 0.18561414726404665


In [ ]:
df.iloc[12]['summary']

['mandela released south african government began transition think felt like part skinner said.',
 'word nelson mandela death spread quickly across united state bringing mix reverence grief man born south africa end belonged world.',
 'honor step shoe nelson mandela portray man defied odds broke barrier championed human right eye world elba said.']

In [ ]:
df.iloc[12]['highlights_clean']

['president obama ordered flag lowered halfstaff memory nelson mandela',
 'mandela loomed large american activist politician',
 'plight repeatedly depicted hollywood']